# nn

> A description here

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#|default_exp nn

In [ ]:
#|export
import random
from neev.engine import Value

In [ ]:
#|export
class Neuron:
    def __init__(self, 
                 nin # number of inputs to the neuron
                ):
        self.w = [Value(random.uniform(-1,1)) for i in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
    def __call__(self,x):
        act =  sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        o = act.tanh()
        return o
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    def __init__(self, 
                 nin,#number of inputs to each neuron in the layer 
                 nout#number of neurons in the layer
                ):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

    
class MLP:
    def __init__(self, 
                 nin,#number of inputs to each neuron in the layer  
                 nouts # list with the number of neurons in each layer of the MLP
                ):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

In [ ]:
#|hide
x = [2.0,3.0]
n = Neuron(2)
n(x)

Value(data=0.9104111583489728)

In [ ]:
#|hide
# we want 3 neurons in our layer, each neuron will take two inputs
l = Layer(2,3)
l(x)

[Value(data=-0.9891878901787617),
 Value(data=-0.9969669092439016),
 Value(data=0.9998805782448378)]

In [ ]:
### |hide
# we want 3 layers in our MLP
# with 4 neurons in the first, 4 in the second
# 1 neuron in the output layer
# each neuron in the MLP will take three inputs
n = MLP(3,[4,4,1])
n(x)

Value(data=-0.9508978458604742)

In [ ]:
#|hide
assert len(n.parameters()) == 4*4 + 4*5 + 5

In [ ]:
#|hide
from neev.viz import view_dot

In [ ]:
#|hide
# view_dot(n(x))

In [ ]:
#|hide
xs = [
    [2.0,3.0,-1.0],
    [3.0,-1.0,0.5],
    [0.5,1.0,1.0],
    [1.0,1.0,-1.0]
]
ys= [1.0,-1.0,-1.0,1.0] #targets

In [ ]:
#|hide
for k in range(10):
    # forward pass
    ypred =[n(x) for x in xs]
#     print(ypred)
    loss = sum((yout-ygt)**2 for ygt,yout in zip(ys,ypred))
    
    # backward pass
    loss.backward()
#     print(n.layers[0].neurons[0].w[0].grad)
#     print(n.layers[0].neurons[0].w[0].data)

    # update
    for p in n.parameters():
        p.data += -0.05 *p.grad
        
    print(f'{k},{loss.data}')

ypred

0,7.687487914931491
1,7.610488174633551
2,7.371873185377382
3,6.587464608845025
4,4.0210897736138165
5,3.066800055356951
6,1.8190917611792388
7,0.2254831016528278
8,0.7823896699101042
9,0.02086395827425442


[Value(data=0.9403886843416586),
 Value(data=-0.9632518211349781),
 Value(data=-0.9781298084240874),
 Value(data=0.8755744584489453)]

#|hide

The training loop above has a bug. Can you spot it?

Essentially we forgot to zero out the gradients!! The gradients continued to accumulate and essentially gave us a huge step size.

In [ ]:
#|hide
n = MLP(3,[4,4,1])

In [ ]:
#|hide
for k in range(10):
    # forward pass
    ypred =[n(x) for x in xs]
    loss = sum((yout-ygt)**2 for ygt,yout in zip(ys,ypred))
    
    # backward pass
    for p in n.parameters():
        p.grad = 0 #zero grad
    loss.backward()

    # update
    for p in n.parameters():
        p.data += -0.05 * p.grad
        p.grad = 0
        
    print(f'{k},{loss.data}')

ypred

0,6.984012446543739
1,5.2990932898900365
2,2.7119446198737363
3,1.1040438399237102
4,0.6116998222981237
5,0.39793797447474055
6,0.28609778803102515
7,0.21982528566041354
8,0.17683287626830044
9,0.14702658201211094


[Value(data=0.8185364247619584),
 Value(data=-0.8521335877093006),
 Value(data=-0.7870031698836025),
 Value(data=0.7835157572695371)]

In [ ]:
#|hide
# view_dot(loss)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()